### Rerebuild
The idea is that if one can rebuild the same program on two different environment and obtain the same result, the program should inherently be reproducible.
We therefore applied this test to a set of 200 known to be not reproducible flatpaks.

In [1]:
%matplotlib notebook

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.widgets import Slider, Button
import json
import os
from helpers import extract_stats
import subprocess
import shutil
from git.repo import Repo
import datetime as dt
from datetime import datetime, timezone

In [2]:
stats_computer_1 = extract_stats('../results/rebuild_twice_on_two_machines//results_computer_1/')
stats_computer_2 = extract_stats('../results/rebuild_twice_on_two_machines/results_computer_2/')
previous_stats = extract_stats('../results/rebuild_half_flathub/')

In [3]:
previous_dict = {stat['name']:stat for stat in previous_stats}

In [4]:
joined_stats = list()
for stat1 in stats_computer_1:
    for stat2 in stats_computer_2:
        if stat1['name'] == stat2['name'] and stat1['build_sucess'] and stat2['build_sucess']:
            joined_stats.append((stat1, stat2))

In [5]:
reproducible = [stat1 for (stat1, stat2) in joined_stats if stat1['rebuild_hash'] == stat2['rebuild_hash']]

In [6]:
print(len(reproducible))

70


In [7]:
bin_reproducible = [stat1 for (stat1, stat2) in joined_stats if stat1['rebuild_bin_hash'] == stat2['rebuild_bin_hash']]

In [8]:
became_bin_reproducible = [stat for stat in bin_reproducible if previous_dict[stat['name']]['is_bin_reproducible'] == False]
were_bin_reproducible = [stat for stat in bin_reproducible if previous_dict[stat['name']]['is_bin_reproducible']]

In [9]:
print(len(bin_reproducible))
print(len(became_bin_reproducible))
print(len(were_bin_reproducible))

73
32
41


In [10]:
build_same_stuff = [stat1 for (stat1, stat2) in joined_stats if stat1['commit'] == stat2['commit']]

In [11]:
print(len(build_same_stuff))

185


In [12]:
theoritical_repro_ratio = len(reproducible)/len(build_same_stuff)
theoritical_bin_repro_ratio = len(became_bin_reproducible)/len(build_same_stuff)
# We can extrapolate to the whole repo where ~41% were reproducible and 58% bin_reproducible
extra_repro_ratio = 0.41 + (1 - 0.41)*theoritical_repro_ratio
extra_bin_repro_ratio = 0.58 + (1 - 0.58)*theoritical_bin_repro_ratio

In [13]:
print(f"We have in theory {theoritical_repro_ratio*100}% of the non-reproducible package that should be reproducible")
print(f"and {theoritical_bin_repro_ratio*100}% of the non-bin-reproducible package that should be bin_reproducible")

We have in theory 37.83783783783784% of the non-reproducible package that should be reproducible
and 17.2972972972973% of the non-bin-reproducible package that should be bin_reproducible


In [14]:
print(f"If we extrapolate with the previous statistics we had, we could have {extra_repro_ratio*100}% reproducible flatpaks")
print(f"and {extra_bin_repro_ratio*100}% bin-reproducible one.")

If we extrapolate with the previous statistics we had, we could have 63.32432432432433% reproducible flatpaks
and 65.26486486486486% bin-reproducible one.
